# Manipulación de datos (preprocesamiento)


In [40]:
# Libraries to handle data
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.datasets import load_wine
df_wine = load_wine(as_frame=True).frame

**Ejercicio 1.** Visualiza la primera fila del dataset.

**Ejercicio 2.** Muestra información general como el número de filas, el número de columnas, el nombre de las columnas y el tipo de dato de cada columna

**Ejercicio 3.** En la columna target, el número 0 corresponde a la etiqueta 'clase1', el 1 a 'clase2' y el 2 a 'clase3'. Modifica el dataset para que aparezcan los nombres en vez de los números. 

**Ejercicio 4.** Elimina la columna flavanoids y en su lugar crea una nueva que se llame graduacion_flavanoids. Esta nueva columna deberá tener como valores alto, medio y bajo dependiendo del valor de la columna flavanoids. 

**Ejercicio 5.** Comprueba el número de ocurrencias por cada una de las clases que queremos predecir. 

**Ejercicio 6.** Balancea el dataset teniendo en cuenta la variable target. Deberá haber el mismo número de ocurrencias de cada uno de los valores de target, para ello deberás añadir nuevas a la que menos tenga y quitar a la que más tenga. 

**Ejercicio 7.** Realiza una codificación one-hot encoding de la variable graduacion_flavanoids 

**Ejercicio 8.** Aplica un filtro para eliminar aquellas ocurrencias en las que 'bajo', 'medio' y 'alto' es igual a 0 o 'bajo', 'medio' y 'alto' es igual a 1. **OJO. no debería haber ninguna.**

**Ejercicio 9.** Muestra qué variables categóricas y numéricas hay.

**Ejercicio 10.** Crea una función que convierta todas las variables categóricas en numéricas aplicando un mapeo. Dicha función devolverá el dataset modificado. Utiliza LabelEncoder. 

**Ejercicio 11.** Normaliza el dataset al completo excepto la columna target.

**Ejercicio 12.** Divide el dataset en tres grupos. El primer grupo será de entrenamiento y tendrá el 80% de los datos. El segundo grupo será de validación y tendrá el 10% de los datos. El tercer grupo será de test y tendrá el 10% de los datos.